<a href="https://colab.research.google.com/github/gguillaux/ml_python/blob/master/NLP_With_News_Folha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of new categories
## 1. Import libraries

In [0]:
import nltk
import requests
import pandas as pd
import requests_html
from bs4 import BeautifulSoup as bs
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

## 2. Define main url, target url and auxiliar functions

In [0]:
def join_text_columns(df):
    '''joins title and subtitle columns'''
    df['text'] = df['title'].str.lower() + ' ' + df['subtitle'].str.lower()
    return df

In [0]:
class Folha:
    def __init__(self):
        self.url = r'https://www.folha.uol.com.br/'
        self.last_news = {'ultimas_noticias' : r'https://www1.folha.uol.com.br/ultimas-noticias/#500'}
        self.categories = self.get_news_categories(self.url)
        self.news_df = self.get_news_dataframe(self.categories)
        self.last_news_df = self.get_news_dataframe(self.last_news)
        

    def get_news_categories(self, url):
        '''get editorials categories from main webpage'''
        s = requests_html.HTMLSession()
        r = s.get(url)
        #r.html.encoding = 'ISO-8859-1'

        soup = bs(r.content.decode('utf-8'), 'html5')
        tags = soup.select('nav ul li.c-site-nav__item a')[1:-3]
        sections = {k.text : '{}#500'.format(k.attrs.get('href')) 
                    if (k.attrs.get('href')[-1] == r'/') 
                    else r'{}/#500'.format(k.attrs.get('href')) for k in tags}
        return sections


    def get_headlines_and_subtitles(self, url, category):
        '''get titles and subtitles of all news in the given category'''
        s = requests_html.HTMLSession()
        r = s.get(url)
        r.html.encoding = r.encoding

        soup = bs(r.content.decode('iso-8859-1'), 'html5')
        tags = soup.select('div.c-headline__content')
        headlines = []
        for k in tags:
            try:
                if len(k.select('p')) > 0:
                    h = {'category': category,
                        'title': k.select('h2')[0].text.strip(),
                        'subtitle': k.select('p')[0].text.strip()}
                    headlines.append(h)
            except Exception as e:
                print("Error on {}.\nMessage = {}".format('k', e))
        return headlines


    def get_news_dataframe(self, categories):
        headlines = [get_headlines_and_subtitles(categories[cat], cat) for cat in categories.keys()]
        frames = [pd.DataFrame(k) for k in headlines]
        df = pd.concat(frames, axis=0)
        return join_text_columns(df)

## 3. Get main categories and structure news as dataframes

In [0]:
folha = Folha()
folha.categories

{'F5': 'https://f5.folha.uol.com.br/#500',
 'coronavírus': 'https://www1.folha.uol.com.br/cotidiano/coronavirus/#500',
 'cotidiano': 'https://www1.folha.uol.com.br/cotidiano/#500',
 'cultura': 'https://www1.folha.uol.com.br/ilustrada/#500',
 'economia': 'https://www1.folha.uol.com.br/mercado/#500',
 'esporte': 'https://www1.folha.uol.com.br/esporte/#500',
 'mundo': 'https://www1.folha.uol.com.br/mundo/#500',
 'opinião': 'https://www1.folha.uol.com.br/opiniao/#500',
 'podcasts': 'https://www1.folha.uol.com.br/podcasts/#500',
 'poder': 'https://www1.folha.uol.com.br/poder/#500'}

In [0]:
sample = get_headlines_and_subtitles(categories['poder'], 'poder')
sample[:5]

[{'category': 'poder',
  'subtitle': 'Ela substituiu o marido, em 2019, na mesma função que ele tinha na legenda até assumir o governo do RJ',
  'title': 'Mulher de Witzel recebeu mais de R$ 350 mil de dinheiro pÃºblico do PSC para atuar como advogada'},
 {'category': 'poder',
  'subtitle': 'Veículos de comunicação com Folha, o UOL e o Grupo Globo suspenderam temporariamente a cobertura no local',
  'title': 'Ataques a jornalistas no Alvorada sÃ£o claro ataque Ã\xa0 liberdade de imprensa, diz Maia'},
 {'category': 'poder',
  'subtitle': 'Procurador-geral da RepÃºblica cuida atualmente de investigaÃ§Ãµes que atingem presidente',
  'title': 'Aras Ã© nome forte para eventual terceira vaga no Supremo, diz Bolsonaro'},
 {'category': 'poder',
  'subtitle': 'A avaliaÃ§Ã£o de sindicalistas Ã© que Doria se mostrou aberto ao diÃ¡logo com as centrais',
  'title': 'SÃ£o Paulo vai defender a democracia atÃ© com medidas judiciais, diz Doria em reuniÃ£o com centrais'},
 {'category': 'poder',
  'subti

## 4. Get headlines

In [0]:
folha.news_df.head()

,category,title,subtitle,text
0,opinião,Sujos e mal lavados,"PF protagoniza operações polêmicas na forma, m...",sujos e mal lavados pf protagoniza operações p...
1,opinião,Abre ou fecha,Datafolha capta apoio a medidas duras mas tamb...,abre ou fecha datafolha capta apoio a medidas ...
2,opinião,O que a Folha pensa,Jornal expressa diariamente seus pontos de vis...,o que a folha pensa jornal expressa diariament...
3,opinião,A hora do vice-presidente,A gravidade da situação atual não admite outra...,a hora do vice-presidente a gravidade da situa...
4,opinião,MÃ£os dadas pela democracia,Cartilha sobre medidas legais orienta jornalis...,mã£os dadas pela democracia cartilha sobre med...


In [0]:
folha.last_news_df.head()

,category,title,subtitle,text
0,ultimas_noticias,EUA acusam Coreia do Norte de lavar dinheiro p...,"Segundo promotores americanos, 28 norte-corean...",eua acusam coreia do norte de lavar dinheiro p...
1,ultimas_noticias,Bolsonaro envia carro de som com mensagem romÃ...,"Em lua de mel comÂ Augusto Aras, o presidente ...",bolsonaro envia carro de som com mensagem romã...
2,ultimas_noticias,Senado aprova linha de crÃ©dito de atÃ© R$ 100...,Medida de ajuda para profissionais liberais du...,senado aprova linha de crã©dito de atã© r$ 100...
3,ultimas_noticias,Paolla Oliveira diz que nÃ£o tem pudor com nud...,Atriz afirma que pessoas tÃªm imagem errada dela,paolla oliveira diz que nã£o tem pudor com nud...
4,ultimas_noticias,Lady Gaga pulsa com euforia em disco que a lev...,"Depois de cantar country, jazz e levar uma est...",lady gaga pulsa com euforia em disco que a lev...


## 4. Define vetorizerer functions

In [0]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
def get_tf_idf_vectorizer(df, ngram):
    vect = TfidfVectorizer(smooth_idf=False, ngram_range=ngram)
    vect.fit(df.text)
    return vect.transform(df.text)

def get_count_vectorizer(df, ngram):
    vect = CountVectorizer(ngram_range=ngram)
    vect.fit(df.text)
    return vect.transform(df.text)

def get_tf_idf_vectorizer_with_stopw(df, ngram):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    vect = TfidfVectorizer(smooth_idf=False, ngram_range=ngram, stop_words=stopwords)
    vect.fit(df.text)
    return vect.transform(df.text)

def get_count_vectorizer_with_stopw(df, ngram):
    stopwords = nltk.corpus.stopwords.words('portuguese')
    vect = CountVectorizer(ngram_range=ngram, stop_words=stopwords)
    vect.fit(df.text)
    return vect.transform(df.text)

In [0]:
def train_decision_tree_classifier(x, y, test_size, rstate):
    X_train, X_test, y_train, y_test = train_test_split(
        x,
        y,
        test_size=test_size,
        stratify=y,
        random_state=42
        )
    tree_classifier = DecisionTreeClassifier()
    tree_classifier.fit(X_train, y_train)
    print('Train Score \t= {}\nTest Score \t= {}'.format(tree_classifier.score(X_train, y_train), 
                                                         tree_classifier.score(X_test, y_test)))
    return tree_classifier

In [0]:
ngrams = [(1,1), (1,2), (2,2), (1,3), (2,3), (3,3)]

In [0]:
# Count Vectorizer 
test_split = 0.3

print('COUNT_VECTORIZER')
print('without stop words')
for ngram in ngrams:
    vect = get_count_vectorizer(folha.news_df, ngram)
    print('NGRAM = {}'.format(ngram))
    _ = train_decision_tree_classifier(vect, folha.news_df.category, test_split, 42)
    print()

print('-' * 50)

print('with stop words')
for ngram in ngrams:
    vect = get_count_vectorizer_with_stopw(folha.news_df, ngram)
    print('NGRAM = {}'.format(ngram))
    _ = train_decision_tree_classifier(vect, folha.news_df.category, test_split, 42)
    print()

COUNT_VECTORIZER
without stop words
NGRAM = (1, 1)
Train Score 	= 0.9075391180654339
Test Score 	= 0.31788079470198677

NGRAM = (1, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.32450331125827814

NGRAM = (2, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.25496688741721857

NGRAM = (1, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3211920529801324

NGRAM = (2, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.2582781456953642

NGRAM = (3, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.18211920529801323

--------------------------------------------------
with stop words
NGRAM = (1, 1)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3443708609271523

NGRAM = (1, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3543046357615894

NGRAM = (2, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.24172185430463577

NGRAM = (1, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3443708609271523

NGRAM = (2, 3)
Train Score 	= 0.9075391180654339
Test Score 	=

In [0]:
# Tf-idf Vectorizer 
test_split = 0.3

print('TF-IDF VECTORIZER')
print('without stop words')
for ngram in ngrams:
    vect = get_tf_idf_vectorizer(folha.news_df, ngram)
    print('NGRAM = {}'.format(ngram))
    _ = train_decision_tree_classifier(vect, folha.news_df.category, test_split, 42)
    print()

print('-' * 50)

print('with stop words')
for ngram in ngrams:
    vect = get_tf_idf_vectorizer_with_stopw(folha.news_df, ngram)
    print('NGRAM = {}'.format(ngram))
    _ = train_decision_tree_classifier(vect, folha.news_df.category, test_split, 42)
    print()

TF-IDF VECTORIZER
without stop words
NGRAM = (1, 1)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3675496688741722

NGRAM = (1, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3576158940397351

NGRAM = (2, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.25165562913907286

NGRAM = (1, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3344370860927152

NGRAM = (2, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.24172185430463577

NGRAM = (3, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.18543046357615894

--------------------------------------------------
with stop words
NGRAM = (1, 1)
Train Score 	= 0.9075391180654339
Test Score 	= 0.36423841059602646

NGRAM = (1, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.33774834437086093

NGRAM = (2, 2)
Train Score 	= 0.9075391180654339
Test Score 	= 0.23178807947019867

NGRAM = (1, 3)
Train Score 	= 0.9075391180654339
Test Score 	= 0.3576158940397351

NGRAM = (2, 3)
Train Score 	= 0.9075391180654339
Test Score 

In [0]:
folha.last_news_df.head()

,category,title,subtitle,text
0,ultimas_noticias,EUA acusam Coreia do Norte de lavar dinheiro p...,"Segundo promotores americanos, 28 norte-corean...",eua acusam coreia do norte de lavar dinheiro p...
1,ultimas_noticias,Bolsonaro envia carro de som com mensagem romÃ...,"Em lua de mel comÂ Augusto Aras, o presidente ...",bolsonaro envia carro de som com mensagem romã...
2,ultimas_noticias,Senado aprova linha de crÃ©dito de atÃ© R$ 100...,Medida de ajuda para profissionais liberais du...,senado aprova linha de crã©dito de atã© r$ 100...
3,ultimas_noticias,Paolla Oliveira diz que nÃ£o tem pudor com nud...,Atriz afirma que pessoas tÃªm imagem errada dela,paolla oliveira diz que nã£o tem pudor com nud...
4,ultimas_noticias,Lady Gaga pulsa com euforia em disco que a lev...,"Depois de cantar country, jazz e levar uma est...",lady gaga pulsa com euforia em disco que a lev...


In [0]:
stopwords = nltk.corpus.stopwords.words('portuguese')
vect = CountVectorizer(ngram_range=(1,1), stop_words=stopwords)
vect.fit(folha.news_df.text)
j = vect.transform(folha.news_df.text)
model = train_decision_tree_classifier(j, folha.news_df.category, 0.3, 42)

Train Score 	= 0.9075391180654339
Test Score 	= 0.34105960264900664


In [0]:
k = vect.transform(folha.last_news_df.text)

In [0]:
model.predict(k[4])

array(['opinião'], dtype=object)

In [0]:
vect.transform(list('eua acusam coreia do norte de lavar dinheiro'))

<44x5575 sparse matrix of type '<class 'numpy.int64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [0]:
folha.last_news_df['token'] = folha.last_news_df['text'].map(word_tokenize)
folha.last_news_df['vect'] = folha.last_news_df['text'].apply(lambda x: vect.transform(list(x)))
folha.last_news_df['category'] = folha.last_news_df['vect'].apply(lambda x:model.predict(x)[0])

In [0]:
folha.last_news_df

,category,title,subtitle,text,token,vect
0,opinião,EUA acusam Coreia do Norte de lavar dinheiro p...,"Segundo promotores americanos, 28 norte-corean...",eua acusam coreia do norte de lavar dinheiro p...,"[eua, acusam, coreia, do, norte, de, lavar, di...",
1,opinião,Bolsonaro envia carro de som com mensagem romÃ...,"Em lua de mel comÂ Augusto Aras, o presidente ...",bolsonaro envia carro de som com mensagem romã...,"[bolsonaro, envia, carro, de, som, com, mensag...",
2,opinião,Senado aprova linha de crÃ©dito de atÃ© R$ 100...,Medida de ajuda para profissionais liberais du...,senado aprova linha de crã©dito de atã© r$ 100...,"[senado, aprova, linha, de, crã©dito, de, atã©...",
3,opinião,Paolla Oliveira diz que nÃ£o tem pudor com nud...,Atriz afirma que pessoas tÃªm imagem errada dela,paolla oliveira diz que nã£o tem pudor com nud...,"[paolla, oliveira, diz, que, nã£o, tem, pudor,...",
4,opinião,Lady Gaga pulsa com euforia em disco que a lev...,"Depois de cantar country, jazz e levar uma est...",lady gaga pulsa com euforia em disco que a lev...,"[lady, gaga, pulsa, com, euforia, em, disco, q...",
...,...,...,...,...,...,...
94,opinião,Lady Gaga derrota Bolsonaro com chÃ¡ de cloroq...,Regina Duarte tambÃ©m aparece em animaÃ§Ã£o fe...,lady gaga derrota bolsonaro com chã¡ de cloroq...,"[lady, gaga, derrota, bolsonaro, com, chã¡, de...",
95,opinião,Roney Ã© flagrado beijando Josefina,Segunda (1) - Tato (Matheus Abreu) discute com...,roney ã© flagrado beijando josefina segunda (1...,"[roney, ã©, flagrado, beijando, josefina, segu...",
96,opinião,Netflix desiste de lanÃ§ar filmes em festivais...,Produtora nÃ£o prestigiarÃ¡ nenhuma mostra nes...,netflix desiste de lanã§ar filmes em festivais...,"[netflix, desiste, de, lanã§ar, filmes, em, fe...",
97,opinião,Vice-PresidÃªncia abre licitaÃ§Ã£o para compra...,"Aparelho serÃ¡ usado por moradores, seguranÃ§a...",vice-presidãªncia abre licitaã§ã£o para compra...,"[vice-presidãªncia, abre, licitaã§ã£o, para, c...",


In [0]:
import numpy as np

In [0]:
np.array('Harvard oferece 96 cursos grÃ¡tis a distÃ¢ncia 	')

array('Harvard oferece 96 cursos grÃ¡tis a distÃ¢ncia \t', dtype='<U48')